# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - można zaimplementować w tym celu wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [187]:
from typing import Tuple, List
import copy
import math

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState

Wielkość planszy

In [188]:
ROW_COUNT = 4
COLUMN_COUNT = 7

In [189]:
class MinMaxSolver:

    def __init__(self, game: ConnectFour):
        self.game = game
        self.fields = game.state.fields
        self.columns_count = len(self.fields) #Trzeba się niezwykle napracować nad taką głupotą
        self.rows_count = len(self.fields[0])
        # print(self.rows_count)
        # print(self.columns_count)
        self.p_min = game.get_players()[0]
        self.p_max = game.get_players()[1]

    def evaluate_position(self, player: Player)->float:
        
        # game1=copy.deepcopy(self.game)
        pass

    def heuristic(self, x, y):
        y_score = math.floor((self.columns_count)/2)-math.floor(abs(y-(self.rows_count-1)/2))
        x_score = math.floor((self.rows_count)/2)-math.floor(abs(x-(self.columns_count-1)/2))
        return x_score+y_score


    def minimax(self, depth, alpha:float, beta:float, is_maximizing_player:bool)-> Tuple[int, float]:
        """Returns column index and score"""
        if self.get_wining_move()[0] != -1 :
            if is_maximizing_player:
                return [self.get_wining_move()[0], 1000]
            else:
                return [self.get_wining_move()[0], -1000]
        # elif (depth == 0):
        #     #heurystyka
        #     return [self.heuristic()]
        #     pass
        
        tab_h = []
        for m in self.game.get_moves():

            game_c=copy.deepcopy(self.game)
            if (depth > 0): #tak jest zrobić heurystykę łatwiej gdyż, wywoływany minmax nie wie jaki ruch właśnie wykonał
                game_c.make_move(m)
                mmsol = MinMaxSolver(game_c)
                print(game_c)
                print(depth)
                # print(mmsol.minimax(depth-1,0,0,(not is_maximizing_player)))
                best_poch = mmsol.minimax(depth-1,0,0,(not is_maximizing_player))
                print(best_poch)
                # tab_h.append(mmsol.minimax(depth-1,0,0,(not is_maximizing_player)))
                tab_h.append([m.column, best_poch[1]])
            else:
                x = m.column
                if  is_maximizing_player: #być może tu powinno być na odwrót
                    return[x, self.heuristic(x, self.get_column_height(x))]
                else:
                    return[x, -1*self.heuristic(x, self.get_column_height(x))]
            
        if(is_maximizing_player):
            #max wartosc z drugiej pozycji krotki
            tab_h.sort(key=lambda a: a[1], reverse=True)
        else:
            tab_h.sort(key=lambda a: a[1], reverse=False)
        return tab_h[0]
    
    def get_column_height(self, col: int) -> int:
        i=0
        for i, field in enumerate(self.fields[col]):
            if field is not None:
                i = i + 1
            else:
                return i
                
    def get_wining_move(self):#returns -1 if wining move do not exist

        moves = self.game.get_moves()
        for m in moves:
            game1=copy.deepcopy(self.game)
            player1 = game1.get_current_player()
            game1.make_move(m)
            # print(game1)
            player2= game1.get_current_player()
            if(game1.get_winner()==player1):
                return [m.column, player1]
            elif(game1.get_winner()==player2):
                return [m.column, player2]
        return [-1, None]
            

    def get_best_move(self)->int:
        pass

    def _get_valid_locations(self)->List[int]:
        pass

    def _is_valid_move(self, col_index:int)->bool:
        pass

TEST HEURYSTYKI

In [190]:
ROW_COUNT = 8
COLUMN_COUNT = 10
p1 = Player("a")
p2 = Player("b")
t = [ [0]*8 for i in range(10)]
game_0 =ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
minmax_0 = MinMaxSolver(game_0)
for i in range(COLUMN_COUNT):
    for j in range(ROW_COUNT):
        t[i][j] = minmax_0.heuristic(i,j)
print(t)
print(t[9][0])

# ROW_COUNT = 4
# COLUMN_COUNT = 4
# p1 = Player("a")
# p2 = Player("b")
# t = [ [0]*4 for i in range(4)]
# game_0 =ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
# minmax_0 = MinMaxSolver(game_0)
# for i in range(COLUMN_COUNT):
#     for j in range(ROW_COUNT):
#         t[i][j] = minmax_0.heuristic(i,j)
# print(t)



[[2, 3, 4, 5, 5, 4, 3, 2], [3, 4, 5, 6, 6, 5, 4, 3], [4, 5, 6, 7, 7, 6, 5, 4], [5, 6, 7, 8, 8, 7, 6, 5], [6, 7, 8, 9, 9, 8, 7, 6], [6, 7, 8, 9, 9, 8, 7, 6], [5, 6, 7, 8, 8, 7, 6, 5], [4, 5, 6, 7, 7, 6, 5, 4], [3, 4, 5, 6, 6, 5, 4, 3], [2, 3, 4, 5, 5, 4, 3, 2]]
2


In [191]:

game = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
game.make_move(ConnectFourMove(4))


print(game.get_winner() == p1)
print(game.get_winner() == p2)
game.make_move(ConnectFourMove(2))
game.make_move(ConnectFourMove(4))
game.make_move(ConnectFourMove(3))
print(game.get_winner() == p1)
print(game.get_winner() == p2)


moves = game.get_moves()
fields = game.state.fields
m1 = moves[0]
print(m1.column)
print(len(moves))
print("col height check")

print(game)

False
False
True
False
0
10
col height check
Current player: b
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ][ ][ ][ ]
[ ][ ][ ][a][b][ ][ ][ ][ ][ ]
[ ][ ][a][a][b][ ][ ][ ][ ][ ]


Rozgrywka

In [192]:
ROW_COUNT = 4
COLUMN_COUNT = 4
p1 = Player("a")
p2 = Player("b")

game1 = ConnectFour(size=(COLUMN_COUNT, ROW_COUNT), first_player=p1, second_player=p2)

game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(2))
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(0))
game1.make_move(ConnectFourMove(1))
game1.make_move(ConnectFourMove(1))

minmax = MinMaxSolver(game1)
# print(game1)
# print(minmax.get_column_height(1))
# print(minmax.get_wining_move())

# sample = [('Jack', 76), ('Beneth', 78), ('Cirus', 77), ('Faiz', 79)]
# sample.sort(key=lambda a: a[1], reverse=True)
# print(sample)

# for m in game1.get_moves():

#             game_c=copy.deepcopy(game1)
#             game_c.make_move(m)
#             print(game_c)
        

# print(game1)
x = minmax.minimax(2,0,0,True)
print(x)

Current player: b
[ ][b][ ][ ]
[ ][a][ ][ ]
[a][a][ ][ ]
[b][a][b][ ]
2
Current player: a
[ ][b][ ][ ]
[b][a][ ][ ]
[a][a][ ][ ]
[b][a][b][ ]
1
[0, -2]
Current player: a
[ ][b][ ][ ]
[ ][a][ ][ ]
[a][a][b][ ]
[b][a][b][ ]
1
[0, -3]
Current player: a
[ ][b][ ][ ]
[ ][a][ ][ ]
[a][a][ ][ ]
[b][a][b][b]
1
[0, -3]
[2, -3]
Current player: b
[ ][b][ ][ ]
[ ][a][ ][ ]
[ ][a][a][ ]
[b][a][b][ ]
2
Current player: a
[ ][b][ ][ ]
[ ][a][ ][ ]
[b][a][a][ ]
[b][a][b][ ]
1
[0, -3]
Current player: a
[ ][b][ ][ ]
[ ][a][b][ ]
[ ][a][a][ ]
[b][a][b][ ]
1
[0, -3]
Current player: a
[ ][b][ ][ ]
[ ][a][ ][ ]
[ ][a][a][ ]
[b][a][b][b]
1
[0, -3]
[0, -3]
Current player: b
[ ][b][ ][ ]
[ ][a][ ][ ]
[ ][a][ ][ ]
[b][a][b][a]
2
Current player: a
[ ][b][ ][ ]
[ ][a][ ][ ]
[b][a][ ][ ]
[b][a][b][a]
1
[0, -3]
Current player: a
[ ][b][ ][ ]
[ ][a][ ][ ]
[ ][a][b][ ]
[b][a][b][a]
1
[0, -3]
Current player: a
[ ][b][ ][ ]
[ ][a][ ][ ]
[ ][a][ ][b]
[b][a][b][a]
1
[0, -3]
[0, -3]
[0, -3]
